In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# #langchain library
# !pip3 install langchain==0.1.0
# #LLM
# !pip3 install accelerate==0.25.0
# !pip3 install transformers==4.36.2
# !pip3 install bitsandbytes==0.41.2
# #Text Embedding
# !pip3 install sentence-transformers==2.2.2
# !pip3 install InstructorEmbedding==1.0.1
# #vectorstore
# !pip3 install pymupdf==1.23.8
# !pip3 install faiss-gpu==1.7.2
# !pip3 install faiss-cpu==1.7.4

# !pip3 install datasets

In [2]:
# model_path = "medical_chatbot.pkl"
# # !cp -R $model_path .

In [3]:
# import torch

In [4]:
# model_id = '/home/koala/AIT second semester/NLP/project/Medical-QA/chatbot/models/fastchat-t5-3b-v1.0'


In [5]:
# from transformers import AutoModelForSeq2SeqLM

# # Define the path to the saved state dictionary file
# file_path = './medical_chatbot_t5_torch_save.pt'

# # Create a new instance of the model architecture
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# # Load the saved state dictionary into the model
# model.load_state_dict(torch.load(file_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

In [6]:
import cloudpickle

def load_model():
    with open('/home/koala/AIT second semester/NLP/project/Medical-QA/chatbot/medical_chatbot_pickle_version3.pkl', 'rb') as f:
        chain = cloudpickle.load(f)
    return chain

In [7]:
model = load_model()

/home/koala/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-29 02:41:34.957260: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 02:41:35.325326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 02:41:36.194448: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
from datasets import load_dataset

# Load the dataset
eval_dataset = load_dataset('KaungHtetCho/plab1_sample')

eval_test = eval_dataset['train'].select(range(2))

In [9]:
eval_test

Dataset({
    features: ['question', 'opa', 'opb', 'opc', 'opd', 'ope', 'cop'],
    num_rows: 2
})

In [1]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Define the function to generate answers for questions along with choices
# def generate_answers_for_questions(dataset, model):
#     answers = []

#     # Initialize TF-IDF vectorizer
#     vectorizer = TfidfVectorizer()

#     # Collect all choices for TF-IDF vectorization
#     all_choices = [example['opa'] + ' ' + example['opb'] + ' ' + example['opc'] + ' ' + example['opd'] + ' ' + example['ope'] for example in dataset]

#     # Fit TF-IDF vectorizer
#     vectorizer.fit(all_choices)

#     for example in dataset:
#         question = example['question']
#         options = [example['opa'], example['opb'], example['opc'], example['opd']]

#         # Construct prompt with question and choices
#         prompt = f"Question: {question}\nOptions: {' | '.join(options)}"

#         # Generate answer using your model
#         generated_answer = model(prompt)

#         # Calculate TF-IDF vectors for generated answer and choices
#         generated_answer_vector = vectorizer.transform([generated_answer['answer']])
#         choices_vectors = vectorizer.transform(options)

#         # Calculate cosine similarity between generated answer and choices
#         cosine_similarities = cosine_similarity(generated_answer_vector, choices_vectors)[0]

#         # Select the choice with the highest cosine similarity
#         max_index = np.argmax(cosine_similarities)
#         nearest_choice = options[max_index]

#         # Append the generated answer and the nearest choice to the list of answers
#         answers.append({
#             "question": question,
#             "choices": options,
#             "generated_answer": generated_answer['answer'],
#             "nearest_choice": nearest_choice
#         })

#     return answers

# # Generate answers for questions along with choices from the dataset
# answers = generate_answers_for_questions(eval_test, model)


In [ ]:
# Format answer function
def format_answer(answer, choices):
    # Replace the '<pad>' tag with an empty string
    cleaned_text = answer.replace('<pad>', '')

    # Split the text into lines, strip leading and trailing whitespace, and join back into a single string
    cleaned_text = '\n'.join(line.strip() for line in cleaned_text.split('\n'))

    # Replace multiple spaces with a single space within each line
    cleaned_text = '\n'.join(' '.join(line.split()) for line in cleaned_text.split('\n'))

    # Concatenate the answer choices with the formatted answer
    formatted_text = f"Answer Choices: {' | '.join(choices)}\n\nGenerated Answer:\n{cleaned_text}"

    return formatted_text

# Loop through answers and compare with ground truth
for i, answer in enumerate(answers):
    # Format the generated answer
    formatted_generated_answer = format_answer(answer['generated_answer'], answer['choices'])

    # Print the question
    print(f"Question {i+1}: {answer['question']}")

    # Print the formatted generated answer
    print(formatted_generated_answer)

    # Get the ground truth answer from the dataset
    cs_answer = answer['nearest_choice']

    ground_truth = eval_test[i]['cop']

    # Print the nearest choice based on BLEU score
    print("Nearest Choice (based on cosine similarity):", cs_answer)
    print('ground truth: ', ground_truth)

    print('-----------------------------')

Question 1: A 55 year old man has diarrhoea and weight loss for 9 months. He has an intensely itchy red rash on his buttocks and thighs. Investigations: Haemoglobin 108 g/L (130–175) Blood film: anisocytosis. Which is the most likely histological appearance on small bowel biopsy?
Answer Choices: Hyperplasia of Brunner’s glands | Increased intraepithelial macrophages | Increased lymphocytes in submucosa | Ulceration

Generated Answer:
Ulceration

Nearest Choice (based on cosine similarity): Ulceration
ground truth:  Villous atrophy
-----------------------------
Question 2: A 26 year old woman is asking for strong pain relief for her abdominal pain and is seen in the Emergency Department. She has no significant past medical history. She looks malnourished. She is restless and irritable. Her temperature is 37.1°C. Which non-prescribed substance is the most important to enquire about in this presentation?
Answer Choices: Alcohol | Amphetamines | Cannabis | Cocaine

Generated Answer:
The mo

In [ ]:
# Initialize a variable to count correct answers
correct_count = 0

# Loop through answers
for i, answer in enumerate(answers):
    # Get the generated answer and ground truth
    cs_answer = answer['nearest_choice']
    ground_truth = eval_test[i]['cop']

    # Check if the generated answer matches the ground truth
    if cs_answer == ground_truth:
        correct_count += 1

# Calculate precision score
precision = correct_count / len(answers)

print('correct_count: ', correct_count)
print('total_question: ', len(answers))

# Print precision score
print("Precision Score:", round(precision, 4))

correct_count:  14
total_question:  50
Precision Score: 0.28
